## Compile Obama's Twitter data and 538 Polling Data

In [74]:
import pandas as pd

In [75]:
polling_df = pd.read_csv('data/obama_polling.csv')
twitter_df = pd.read_csv('data/obama_twitter.csv')
polling_df.drop(["no_data", "disapproving"], axis=1, inplace=True)

In [76]:
polling_df.head()

,start_date,end_date,approving
0,1/16/17,1/19/2017,59
1,1/9/17,1/15/2017,57
2,1/2/17,1/8/2017,55
3,12/26/16,1/1/2017,55
4,12/19/16,12/25/2016,56


I want only one date column, so I want to extend the range of start_date to end_date 

In [77]:
polling_df['row'] = range(len(polling_df))
starts = polling_df[['start_date', 'approving', 'row']].rename(columns={'start_date':'date'})
ends = polling_df[['end_date', 'approving', 'row']].rename(columns={'end_date':'date'})
df_decomp = pd.concat([starts, ends])
df_decomp = df_decomp.set_index('row', append=True)
df_decomp.sort_index()


date  approving
    row                      
0   0      1/16/17         59
    0    1/19/2017         59
1   1       1/9/17         57
    1    1/15/2017         57
2   2       1/2/17         55
...            ...        ...
415 415   2/8/2009         65
416 416    1/26/09         66
    416   2/1/2009         66
417 417    1/19/09         67
    417  1/25/2009         67

[836 rows x 2 columns]

In [78]:
df_decomp.date = pd.to_datetime(df_decomp.date)
df_decomp = df_decomp.groupby(['row']).apply(lambda x: x.set_index('date').resample('D').fillna(method='pad'))
df_decomp = df_decomp.reset_index(['row'], drop=True)


In [96]:
df_decomp.sort_values(by=['date'], inplace=True, ascending=False)
df_decomp.head()
df_decomp.to_csv('data/obama_polling.csv')

In [94]:
twitter_df.date = pd.to_datetime(twitter_df.date)
twitter_df['date'] = twitter_df['date'].dt.strftime('%Y-%m-%d')
twitter_df.head()

,date,replies,retweets,favorites
0,2017-10-16,17064,89916,641842
1,2017-10-02,21588,405895,1715753
2,2017-09-30,8476,60651,503255
3,2017-09-26,10657,113807,334901
4,2017-09-25,3886,58449,328106


In [97]:
polling_df = pd.read_csv('data/obama_polling.csv')
polling_df.head()

,date,approving
0,2017-01-19,59
1,2017-01-18,59
2,2017-01-17,59
3,2017-01-16,59
4,2017-01-15,57


In [100]:
final_df = pd.merge(twitter_df, polling_df, on="date")
final_df.head()
final_df.to_csv('data/obama_aggregate.csv')